In [1]:
import re
import spacy
from spacy.util import minibatch, compounding
import random

In [30]:
x = [
    # "According to Rutva, the african bank closes at 3 p.m.", #{'entities': [(24, 36, 'BANK')]}),
    # "Brinda Nathwani will make the payment from my african bank account.", #{'entities': [(46, 58, 'BANK')]}),
    # #"Nihar Thakker deposited a lot of money in the african account.", #{'entities': [(46, 53, 'BANK')]}),
    # #"Saumil Trivedi went to the african and changed his currency.", #{'entities': [(27, 34, 'BANK')]}),
    # #"Please transfer 199 rand using eft from african account.", #{'entities': [(40, 47, 'BANK')]}),
    # "Jiya Patel wants to work at the african bank.", #{'entities': [(32, 44, 'BANK')]}),  
     'marchantile '
     ]
for i in x:
    regex_num = re.compile('marchantile')
    s = regex_num.search(i) 
    print('Starting Position: ', s.start()) 
    print('Ending Position: ', s.end()) 
    print(i[s.start():s.end()])

Starting Position:  0
Ending Position:  11
marchantile


In [26]:
custom = [('according to rutva, the african bank closes at 3 p.m.',{'entities': [(24, 36, 'BANK')]}),
 ('brinda nathwani will make the payment from my african bank account number 0192837465.',{'entities': [(46, 58, 'BANK')]}),
 ('nihar thakker deposited a lot of money in the african account number 1234567890 .',{'entities': [(46, 53, 'BANK')]}),
 ('saumil trivedi went to the african and changed his currency.',{'entities': [(27, 34, 'BANK')]}),
 ('please transfer 199 rand using eft from african account.',{'entities': [(40, 47, 'BANK')]}),
 ('jiya patel wants to work at the african bank.',{'entities': [(32, 44, 'BANK')]}),
 ('african bank',{'entities': [(0, 12, 'BANK')]}),
 ('african',{'entities': [(0, 7, 'BANK')]}),

 ('absa bank',{'entities': [(0, 9, 'BANK')]}),
 ('absa',{'entities': [(0, 4, 'BANK')]}),
 ('vineet patel works at the absa bank.', {'entities': [(26, 36, 'BANK')]}),
 ('please withdrawl 67 rand using eft from absa bank account number 3527810327.',{'entities': [(40, 49, 'BANK')]}),
 ('i would like to withdrawal 10 rand using eft from my absa bank account.',{'entities': [(53, 62, 'BANK')]}),
 ('i want to withdrawal 109 rand from my absa account number 7810263486.',{'entities': [(38, 42, 'BANK')]}),
 ('rushin  patel would like to withdrawal 101 rand using eft from my absa account.',{'entities': [(66, 70, 'BANK')]}),
 ('please withdrawl 67 rand using eft from absa account number 0987654321',{'entities': [(40, 44, 'BANK')]}),
 
 ('marchantile bank',{'entities': [(0, 16, 'BANK')]}),
 ('marchantile ',{'entities': [(0, 11, 'BANK')]}),
 ('the town of barwani is situated near the marchantile bank',{'entities': [(41, 57, 'BANK')]}),
 ('i would like to withdraw 201 rand from marchantile account number 2893456107.',{'entities': [(39, 50, 'BANK')]}),
 ('please withdrawl 67 rand using eft from marchantile account',{'entities': [(40, 51, 'BANK')]}),
 ('i would like to withdraw two hundred one rand from marchantile account using eft',{'entities': [(51, 62, 'BANK')]}),
 ('according to rutva, the marchantile bank closes at 3 p.m.',{'entities': [(24, 40, 'BANK')]}),
 ('want to withdrawal 109 rand from my marchantile bank account number 8620154295',{'entities': [(36, 52, 'BANK')]}),
 

]

In [34]:
#Loading spacy model
nlp=spacy.load('en_core_web_sm') 

# Getting the pipeline component
ner = nlp.get_pipe("ner")

# Adding new entities
for _, annotations in custom:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
    #print(ent)

# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

    # Training for 200 iterations
    for iteration in range(200):
        # shuufling examples  before every iteration
        random.shuffle(custom)
        losses = {}

        # batching up the examples using spaCy's minibatch
        batches = minibatch(custom, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, drop=0.5,  losses=losses)
            #print("Losses", losses)

In [39]:
# Testing the model with created list of sentences
text = ["marchantile bank"]
doc = nlp(str(text))
Entities = [(ent.text, ent.label_) for ent in doc.ents]
Entities

[('marchantile bank', 'BANK')]

In [42]:
from pathlib import Path
output_dir = Path('/content/drive/MyDrive/Chatbot/Rutva/bank_custom_ner/')
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'bank_ner_model'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/drive/MyDrive/Chatbot/Rutva/bank_custom_ner


In [49]:
output_dir = '/content/drive/MyDrive/Chatbot/Rutva/bank_custom_ner/'
nlp2 = spacy.load(output_dir)
doc = nlp2("i would like to withdraw 201 rand from absa bank account number 2893456107.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('absa bank', 'BANK')]
